In [1]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install typing-extensions


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade typing-extensions


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF

def extract_labels(labels):
    # Map emotion labels to integers
    label_mapping = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
    return np.array([label_mapping[label] for label in labels])

class EmotionDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.data, self.labels = self.load_data(dataset_path)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
    @staticmethod
    def load_data(dataset_path):
        data = []
        labels = []

        for emotion_folder in os.listdir(dataset_path):
            emotion_path = os.path.join(dataset_path, emotion_folder)
            for img_name in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path)
                data.append(img)
                labels.append(emotion_folder)

        return np.array(data), extract_labels(labels)
    
    


In [7]:
if torch.backends.mps.is_available():
	device=torch.device("mps")
elif torch.cuda.is_available():
	device=torch.device("cuda")
else:
	device=torch.device("cpu")

print(device)

cuda


In [8]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
train_path = 'FER2013/train'
absolute_train_path = os.path.abspath(train_path)

test_path = 'FER2013/test'
absolute_test_path = os.path.abspath(test_path)

train_dataset = EmotionDataset(absolute_train_path, transform=transform)
test_dataset = EmotionDataset(absolute_test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [10]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained VGG model
class ModifiedVGG(nn.Module):
    def __init__(self, num_classes=7):
        super(ModifiedVGG, self).__init__()
        vgg16 = models.vgg16(pretrained=True)
        
        # Remove the last fully connected layer and add your own
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        
        # Modify the classifier to match your number of classes
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Instantiate the modified VGG model
model = ModifiedVGG(num_classes=7).to(device)

/home/mahadevarao.s/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mahadevarao.s/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/mahadevarao.s/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:11<00:00, 46.9MB/s] 


In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
import torch.optim as optim

# Assuming you have a GPU available, check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move your model to the GPU
model.to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        # Move inputs and labels to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Convert labels to torch.long
        labels = labels.long()
        
        # Convert outputs to torch.long if necessary
        # outputs = outputs.long()  # Uncomment this line if needed
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


Epoch 1/10, Loss: 1.8641163110733032
Epoch 2/10, Loss: 1.8295894861221313
Epoch 3/10, Loss: 1.8389605283737183
Epoch 4/10, Loss: 1.8650093078613281
Epoch 5/10, Loss: 1.8706254959106445
Epoch 6/10, Loss: 2.0830509662628174
Epoch 7/10, Loss: 1.8404502868652344
Epoch 8/10, Loss: 2.0572593212127686
Epoch 9/10, Loss: 1.8344014883041382
Epoch 10/10, Loss: 2.1120951175689697


In [13]:
for inputs, labels in train_loader:
    print(type(inputs))
    print(type(labels))
    print(inputs)
    print(labels)
    break

<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([[[[-0.7822, -0.7822, -0.7822,  ..., -1.4158, -1.4158, -1.4158],
          [-0.7822, -0.7822, -0.7822,  ..., -1.4158, -1.4158, -1.4158],
          [-0.7822, -0.7822, -0.7822,  ..., -1.4158, -1.4158, -1.4158],
          ...,
          [ 2.1633,  2.1633,  2.1633,  ...,  0.0569,  0.0569,  0.0569],
          [ 2.1633,  2.1633,  2.1633,  ...,  0.0569,  0.0569,  0.0569],
          [ 2.1633,  2.1633,  2.1633,  ...,  0.0569,  0.0569,  0.0569]],

         [[-0.6702, -0.6702, -0.6702,  ..., -1.3179, -1.3179, -1.3179],
          [-0.6702, -0.6702, -0.6702,  ..., -1.3179, -1.3179, -1.3179],
          [-0.6702, -0.6702, -0.6702,  ..., -1.3179, -1.3179, -1.3179],
          ...,
          [ 2.3410,  2.3410,  2.3410,  ...,  0.1877,  0.1877,  0.1877],
          [ 2.3410,  2.3410,  2.3410,  ...,  0.1877,  0.1877,  0.1877],
          [ 2.3410,  2.3410,  2.3410,  ...,  0.1877,  0.1877,  0.1877]],

         [[-0.4450, -0.4450, -0.4450,  ..., -1.0898, -1.

In [14]:
model.eval()
test_accuracy = 0  
num_correct = 0
num_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        test_accuracy += acc

print(f'Test Accuracy: {test_accuracy / len(test_loader):.4f}')

Test Accuracy: 0.2091
